<a href="https://colab.research.google.com/github/greypen2006/Final_Project/blob/main/N3.%20Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Merging Indicator Data**

In [ ]:
#This tells python to draw the graphs "inline" - in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
import statistics
import seaborn as sns
from IPython.display import display, Math, Latex, display_latex
import plotly.express as px
import pylab
import pandas as pd
import numpy as np
# make the plots (graphs) a little wider by default
pylab.rcParams['figure.figsize'] = (10., 8.)
sns.set(font_scale=1.5)
sns.set_style("white")

  **Outline**

1. Clean and identify indeces to merge data with
2. Upload all csv for indicators to use in the OLS
3. Create and download a new merged df that can be used for future analysis.

In [ ]:
#First upload all the data from the Google drive csv folder for each indicator (easier to upload all then create a shortcut)
from google.colab import files
uploaded=files.upload()


Saving merged_data with 24 pub count.csv to merged_data with 24 pub count.csv


In [ ]:
#opening the files to find similar columns to use as indeces
sex_df=pd.read_csv('Population.csv', skiprows=35, nrows=13)
sex_df.head()

In [ ]:
rent_df=pd.read_csv('rent_prices.csv')
rent_df.head()

In [ ]:
income_df=pd.read_csv('weekly_income_cleaned.csv')
income_df.head()

In [ ]:
pub_df=pd.read_csv('PubCount(Sheet1).csv')
pub_df.head()

In [ ]:
crime_df=pd.read_csv('crime_cleaned_final.csv')
crime_df.head()

In [ ]:
sexual_crime_df = crime_df[crime_df['MajorText'] == 'SEXUAL OFFENCES']
sexual_crime_df.head()

In [ ]:
job_df=pd.read_csv('unemployment_cleaned.csv')
job_df.head()
#

# Merging Data
`sex_df`, `rent_df`, `income_df`, `pub_df`, `sexual_crime_df`, and `job_df` need to be merged into a single master dataframe: `merged_df` to carry out OLS.

*   **`sex_df`**:
    *   Rename 'Borough' to 'BoroughName' and 'All ages' to 'Female_Population'.
    *  Assign the columns denoting populations in 2017-2024 under 'Year'
*   **`rent_df`**:
    *   Rename 'Borough' to 'BoroughName'.
    *   Melt the yearly rent columns (e.g., '2025-2024') into two new columns: 'Year_Range' and 'Rent_Price'.
    *   Extract the end year from 'Year_Range' (e.g., '2024' from '2025-2024') and convert it to an integer, assigning this to a new 'Year' column.

*   **`income_df`**:
    *   Melt the borough-specific income columns (all columns except 'Year') into 'BoroughName' and 'Weekly_Income', retaining the 'Year' column.

*   **`pub_df`**:
    *   Melt the year columns (e.g., '2017') into a 'Year' column and a 'Pub_Count' column, keeping 'BoroughName'.

*   **`sexual_crime_df`**:
    *   Group by 'BoroughName' and 'Year'.
    *   Sum the 'CrimeCount' for each group to get the 'Sexual_Crime_Count' for each borough and year.

*   **`job_df`**:
    *   Rename 'local authority: district / unitary (as of April 2023)' to 'BoroughName'.
    *   Melt the year range columns (e.g., 'Jul 2016-Jun 2017') along with their corresponding 'Conf' columns.
    *   Extract the end year from the melted year ranges (e.g., '2017' from 'Jul 2016-Jun 2017') and convert it to an integer, assigning this to a new 'Year' column.
    *   Drop the 'Conf' columns and rename the value column from the melt operation to 'Unemployment_Rate'.

After preparing each dataframe, perform a series of inner merges on 'BoroughName' and 'Year' to combine them into a single `merged_df`.

Finally, from `merged_df`, select and rename the following columns:
*   'Female_Population' to 'femalepop'
*   'Rent_Price' to 'rent price'
*   'Weekly_Income' to 'weekly income'
*   'Dominant_Ethnicity' to 'ethnicity'
*   'Pub_Count' to 'pub count'
*   'Sexual_Crime_Count' to 'sexual crime'
*   'Unemployment_Rate' to 'unemployment'

Display the head and information (`.info()`) of the final `merged_df`.

In [ ]:
# Rename the 'local authority...' column to 'BoroughName'
sex_df = sex_df.rename(columns={'local authority: county / unitary (as of April 2023)': 'BoroughName'})

# Identify columns that represent years for melting
year_cols = [str(year) for year in range(2017, 2025)] # Covers 2017-2024 as per sex_df.head()

# Melt the DataFrame to transform year columns into a single 'Year' column and 'Female_Population'
sex_df_melted = sex_df.melt(
    id_vars=['BoroughName', 'mnemonic'], # Keep BoroughName and mnemonic as identifier variables during melt
    value_vars=year_cols,
    var_name='Year',
    value_name='Female_Population'
)

# Convert the 'Year' column to integer type
sex_df_melted['Year'] = sex_df_melted['Year'].astype(int)

# Update sex_df to be the melted version, dropping the 'mnemonic' column as it's not needed for the final merge
sex_df = sex_df_melted.drop(columns=['mnemonic'])

sex_df.head()

In [ ]:
# Replace the boroughs to BoroughName for consistency
rent_df = rent_df.rename(columns={'Borough': 'BoroughName'})
rent_df.head()

In [ ]:
rent_df_melted = rent_df.melt(id_vars=['BoroughName'],
                               value_vars=[col for col in rent_df.columns if '-' in col],
                               var_name='Year_Range',
                               value_name='Rent_Price')
rent_df_melted.head()

In [ ]:
# Extract end year from year range and convert to an integer to store as the Year column, replacing year range with this
rent_df_melted['Year'] = rent_df_melted['Year_Range'].apply(lambda x: int(x.split('-')[1]))
rent_df_melted = rent_df_melted.drop(columns=['Year_Range'])
rent_df_melted.head()

In [ ]:
# Melt borough specific income into boroughname and income per year
income_df_melted = income_df.melt(id_vars=['Year'], var_name='BoroughName', value_name='Weekly_Income')
income_df_melted.head()

In [ ]:
# Transform year columns into 'Year' column and 'Pub Count' column corresponding to BoroughName
pub_df_melted = pub_df.melt(id_vars=['BoroughName'], var_name='Year', value_name='Pub_Count')
pub_df_melted['Year'] = pub_df_melted['Year'].astype(int)
pub_df_melted.head()

In [ ]:
# Group by BoroughName and Year as these are the constants throughout the dfs, summing the crime rate for each group
sexual_crime_df_agg = sexual_crime_df.groupby(['BoroughName', 'Year'])['CrimeRate'].sum().reset_index()
sexual_crime_df_agg = sexual_crime_df_agg.rename(columns={'CrimeRate': 'Sexual_Crime_Rate'})
sexual_crime_df_agg.head()

In [ ]:
# Rename for consistency as done above
job_df = job_df.rename(columns={'local authority: district / unitary (as of April 2023)': 'BoroughName'})
job_df.head()

In [ ]:
#Identify unemployment rate columns, exclude Conf columns and melt job_df to transform columns to Year Range and Unemployment rate corresponding to Borough Names
unemployment_cols = [col for col in job_df.columns if 'Jul' in col and 'Conf' not in col]
job_df_melted = job_df.melt(id_vars=['BoroughName', 'mnemonic'],
                            value_vars=unemployment_cols,
                            var_name='Year_Range',
                            value_name='Unemployment_Rate')
job_df_melted.head()

In [ ]:
# Extract end year of year range and assign to new Year column as done above
job_df_melted['Year'] = job_df_melted['Year_Range'].apply(lambda x: int(x.split('-')[1].split(' ')[1]))
job_df_melted.head()

In [ ]:
# Drop mnemonic and Year Range
job_df_melted = job_df_melted.drop(columns=['Year_Range', 'mnemonic'])
job_df_melted.head()

In [ ]:
import pandas as pd
from itertools import product

# Clean 'BoroughName' in each dataframe by stripping whitespace
sex_df['BoroughName'] = sex_df['BoroughName'].str.strip()
rent_df_melted['BoroughName'] = rent_df_melted['BoroughName'].str.strip()
income_df_melted['BoroughName'] = income_df_melted['BoroughName'].str.strip()
race_df_dominant_ethnicity['BoroughName'] = race_df_dominant_ethnicity['BoroughName'].str.strip()
pub_df_melted['BoroughName'] = pub_df_melted['BoroughName'].str.strip()
sexual_crime_df_agg['BoroughName'] = sexual_crime_df_agg['BoroughName'].str.strip()
job_df_melted['BoroughName'] = job_df_melted['BoroughName'].str.strip()

# Consolidate all unique BoroughNames from all dataframes
all_boroughs = pd.concat([
    sex_df['BoroughName'],
    rent_df_melted['BoroughName'],
    income_df_melted['BoroughName'],
    race_df_dominant_ethnicity['BoroughName'],
    pub_df_melted['BoroughName'],
    sexual_crime_df_agg['BoroughName'],
    job_df_melted['BoroughName']
]).unique()

# Filter out non-borough entries from all_boroughs
non_borough_entries = ['Column Total', 'Population estimates - local authority based by five year age band'] # Add any other identified non-borough entries
all_boroughs = [b for b in all_boroughs if b not in non_borough_entries]

# Define the years to include
years_to_include = pd.Series(range(2017, 2025)) # 2017 to 2024 inclusive

# Create a master dataframe with all borough-year combinations
master_combinations = pd.DataFrame(list(product(all_boroughs, years_to_include)), columns=['BoroughName', 'Year'])

# Start merged_df with the master combinations
merged_df = master_combinations

# Perform left merges with each prepared dataframe
merged_df = pd.merge(merged_df, sex_df, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, rent_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, income_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, race_df_dominant_ethnicity, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, pub_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, sexual_crime_df_agg, on=['BoroughName', 'Year'], how='left')

# Re-extract the end year from 'Year_Range' as the kernel state indicates it's missing
job_df_melted['Year'] = job_df_melted['Year_Range'].apply(lambda x: int(x.split('-')[1].split(' ')[1]))
# Re-drop the 'Year_Range' and 'mnemonic' columns
job_df_melted = job_df_melted.drop(columns=['Year_Range', 'mnemonic'])

merged_df = pd.merge(merged_df, job_df_melted, on=['BoroughName', 'Year'], how='left')

# Rename columns as specified in the task
merged_df = merged_df.rename(columns={
    'Female_Population': 'femalepop',
    'Rent_Price': 'rent price',
    'Weekly_Income': 'weekly income',
    'Dominant_Ethnicity': 'ethnicity',
    'Pub_Count': 'pub count',
    'Sexual_Crime_Rate': 'sexual crime', # Corrected column name for sexual crime
    'Unemployment_Rate': 'unemployment'
})

# Display head and info of the final merged_df
display(merged_df.head())
display(merged_df.info())

In [ ]:
merged_df.tail()

In [ ]:
#Convert to csv and download
merged_df.to_csv('merged_data with 24 pub count.csv')

In [ ]:
files.download('merged_data with 24 pub count.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>